# Tax Consultation Workflow Testing

This notebook allows you to test and develop the science workflow interactively.

**Owner**: Science Team  
**Purpose**: Test LangGraph workflow, prompts, and knowledge base integration

## 1. Setup & Imports

In [ ]:
import sys
import os
from pathlib import Path
import json
import asyncio

# Add backend to path
backend_path = Path.cwd()
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

print(f"✓ Backend path: {backend_path}")
print(f"✓ Python path configured")

In [ ]:
# Import science modules
from science.services.knowledge_parser import KnowledgeBaseParser, parse_knowledge_base
from science.agents.workflow import TaxConsultationWorkflow
from science.config import science_config

print("✓ Science modules imported successfully")
print(f"✓ Using AI provider: {science_config.AI_MODEL_PROVIDER}")
print(f"✓ Model: {science_config.OPENAI_MODEL if science_config.AI_MODEL_PROVIDER == 'openai' else science_config.GEMINI_MODEL}")

## 2. Parse Knowledge Base

First, we need to parse the markdown files from `tax_team/knowledge_base/` into JSON format.

In [ ]:
# Parse knowledge base
print("📁 Parsing knowledge base...")
kb = parse_knowledge_base()

print("\n✓ Knowledge base parsed successfully!")
print(f"\n📊 Statistics:")
print(f"  - Gating questions: {len(kb['intake']['gating_questions']['questions'])}")
print(f"  - Modules: {len(kb['intake']['modules'])}")
print(f"  - Tags: {len(kb['tags']['tag_definitions'])}")

print(f"\n💾 Cached JSON files created in: science/knowledge_cache/")

### View Sample Data

In [ ]:
# View first gating question
sample_q = kb['intake']['gating_questions']['questions'][0]
print("📝 Sample Gating Question:")
print(f"  ID: {sample_q['id']}")
print(f"  Question: {sample_q['question']}")
print(f"  Action: {sample_q['action']}")

In [ ]:
# View first tag definition
tag_id = list(kb['tags']['tag_definitions'].keys())[0]
sample_tag = kb['tags']['tag_definitions'][tag_id]
print("🏷️  Sample Tag:")
print(f"  ID: {sample_tag['id']}")
print(f"  Name: {sample_tag['name']}")
print(f"  Description: {sample_tag['description'][:100]}...")
print(f"  Jurisdictions: {list(sample_tag['forms'].keys())}")

## 3. Initialize Workflow

In [ ]:
# Create workflow instance
workflow = TaxConsultationWorkflow()
print("✓ Workflow initialized")
print(f"  - Recursion limit: {science_config.WORKFLOW_RECURSION_LIMIT}")
print(f"  - Min tags for transition: {science_config.MIN_TAGS_FOR_TRANSITION}")
print(f"  - Min conversation length: {science_config.MIN_CONVERSATION_LENGTH}")

## 4. Start Consultation

Start a new tax consultation session.

In [ ]:
# Start consultation with initial message
initial_message = "I'm a U.S. citizen living in Canada and need help with my taxes."

result = await workflow.start_consultation(initial_message)

session_id = result['session_id']
print(f"🆔 Session ID: {session_id}")
print(f"\n📍 Phase: {result['current_phase']}")
print(f"\n🤖 Agent Response:")
print(f"   {result['message']}")
print(f"\n💬 Quick Replies: {result['quick_replies']}")
print(f"\n🏷️  Tags: {result['assigned_tags']}")

## 5. Continue Conversation

Continue the conversation by sending user responses.

In [ ]:
# Helper function to send message
async def send_message(message: str):
    result = await workflow.continue_consultation(session_id, message)
    
    print(f"👤 User: {message}")
    print(f"\n🤖 Agent: {result['message']}")
    print(f"\n💬 Quick Replies: {result['quick_replies']}")
    print(f"🏷️  Tags: {result['assigned_tags']}")
    print(f"📍 Phase: {result['current_phase']}")
    
    if result.get('transition'):
        print(f"\n⚡ TRANSITION: Moving to forms analysis!")
    
    print("\n" + "="*80 + "\n")
    return result

In [ ]:
# Answer the first question
result = await send_message("Yes, I am a U.S. citizen")

In [ ]:
# Continue answering questions
result = await send_message("Yes, I am a Canadian tax resident")

In [ ]:
# Another response
result = await send_message("Yes, I earn employment income in Canada")

In [ ]:
# Keep answering...
result = await send_message("Yes, I work for a Canadian company full-time")

In [ ]:
# More questions...
result = await send_message("No, I don't have rental property")

## 6. View Session State

Inspect the current state of the consultation.

In [ ]:
# Get session summary
summary = await workflow.get_session_summary(session_id)

print("📊 Session Summary:")
print(f"\n  🆔 Session ID: {summary['session_id']}")
print(f"  📍 Phase: {summary['current_phase']}")
print(f"  📝 Module: {summary.get('current_module', 'N/A')}")
print(f"  💬 Messages: {summary['message_count']}")
print(f"  🏷️  Tags: {summary['assigned_tags']}")
print(f"  ✅ Completed Modules: {summary['completed_modules']}")
print(f"  ⚡ Ready for forms analysis: {summary['has_sufficient_tags']}")

In [ ]:
# View conversation history
history = await workflow.get_conversation_history(session_id)

print("📜 Conversation History:\n")
for i, msg in enumerate(history, 1):
    role = "👤 User" if msg['role'] == 'user' else "🤖 Agent"
    print(f"{i}. {role}: {msg['content'][:100]}...")
    print()

## 7. Force Forms Analysis

If you have enough information, you can force transition to forms analysis.

In [ ]:
# Force forms analysis
result = await workflow.force_forms_analysis(session_id)

if 'error' in result:
    print(f"❌ Error: {result['error']}")
else:
    print("✓ Forms analysis triggered!\n")
    print("📋 Forms Analysis Report:\n")
    print(result['message'])
    print("\n" + "="*80)
    
    if result.get('forms_analysis'):
        analysis = result['forms_analysis']
        print(f"\n📊 Complexity: {analysis.get('estimated_complexity', 'N/A')}")
        print(f"\n📄 Forms Required: {len(analysis.get('required_forms', []))}")
        print(f"\n💡 Recommendations: {len(analysis.get('recommendations', []))}")

## 8. Debug Utilities

Helper functions for debugging and inspection.

In [ ]:
# Get detailed debug information
debug_info = await workflow.debug_session(session_id)

print("🔍 Debug Information:\n")
print(json.dumps(debug_info, indent=2))

In [ ]:
# View all available tags in knowledge base
print("🏷️  All Available Tags:\n")
for tag_id, tag_def in kb['tags']['tag_definitions'].items():
    print(f"  - {tag_id}: {tag_def['name']}")

In [ ]:
# View all modules
print("📚 All Modules:\n")
for module_id, module_data in kb['intake']['modules'].items():
    print(f"  - {module_id}: {module_data['title']}")
    print(f"    Questions: {len(module_data['questions'])}")
    print()

## 9. Test Different Scenarios

Create multiple test scenarios to validate workflow behavior.

In [ ]:
# Scenario 1: US citizen with Canadian employment
async def test_scenario_1():
    print("🧪 Test Scenario 1: US Citizen with Canadian Employment\n")
    
    result = await workflow.start_consultation("I'm a US citizen working in Canada")
    s_id = result['session_id']
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "Yes, I am a US citizen")
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "Yes, I am a Canadian resident")
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "Yes, I earn employment income in Canada")
    print(f"Agent: {result['message']}\n")
    
    summary = await workflow.get_session_summary(s_id)
    print(f"\n✓ Tags assigned: {summary['assigned_tags']}")
    print(f"✓ Phase: {summary['current_phase']}")
    
    return s_id

# Run scenario
scenario_1_id = await test_scenario_1()

In [ ]:
# Scenario 2: Canadian with US rental property
async def test_scenario_2():
    print("🧪 Test Scenario 2: Canadian with US Rental Property\n")
    
    result = await workflow.start_consultation("I'm Canadian and own rental property in the US")
    s_id = result['session_id']
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "No, I'm not a US citizen")
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "Yes, I am a Canadian resident")
    print(f"Agent: {result['message']}\n")
    
    result = await workflow.continue_consultation(s_id, "Yes, I own rental property in the US")
    print(f"Agent: {result['message']}\n")
    
    summary = await workflow.get_session_summary(s_id)
    print(f"\n✓ Tags assigned: {summary['assigned_tags']}")
    print(f"✓ Phase: {summary['current_phase']}")
    
    return s_id

# Run scenario
scenario_2_id = await test_scenario_2()

## 10. Reset and Start Fresh

If you want to start over, just create a new workflow instance or start a new consultation.

In [ ]:
# Start fresh consultation
print("🔄 Starting fresh consultation...\n")
result = await workflow.start_consultation("Hello, I need tax help")
session_id = result['session_id']
print(f"✓ New session started: {session_id}")
print(f"\n🤖 Agent: {result['message']}")

---

## Tips for Development

1. **Modify prompts**: Edit `science/agents/prompts.py` and restart the kernel
2. **Change workflow logic**: Edit `science/agents/nodes.py` or `workflow.py` and restart kernel
3. **Update questions**: Edit `tax_team/knowledge_base/intake/questions.md` and re-run the parser cell
4. **Update tags**: Edit `tax_team/knowledge_base/tags/definitions.md` and re-run the parser cell
5. **Debug**: Use `debug_session()` and `get_session_summary()` to inspect internal state

Happy testing! 🚀